<a href="https://colab.research.google.com/github/noelakkidas/Plant-disease-classification/blob/main/palnt_disease_image_classifier_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the required libraries and packages

In [ ]:
from tensorflow.keras import models
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg19 import VGG19


Collecting the data through Kaggle Website directly to Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle

In [ ]:
%cd /content/drive/MyDrive/plant_disease

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/plant_disease"

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset 

In [ ]:
!unzip new-plant-diseases-dataset.zip

In [ ]:
%cd /content/drive/MyDrive/plant_disease

Getting the data

In [ ]:
os.listdir("/content/drive/MyDrive/plant_disease/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)")


In [ ]:
os.li

In [ ]:
train_dir = ImageDataGenerator(zoom_range =0.5,shear_range = 0.3,rescale =1./255, horizontal_flip = True)
valid_dir = ImageDataGenerator(rescale = 1./255)

In [ ]:
train = train_dir.flow_from_directory(directory = "/content/drive/MyDrive/plant_disease/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",target_size = (256,256),batch_size = 32 )
valid = valid_dir.flow_from_directory(directory = "/content/drive/MyDrive/plant_disease/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",target_size = (256,256),batch_size = 32 )

Generating data

In [ ]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

# Generator for our training data
train_image_generator = ImageDataGenerator(rescale=1./255)
# Generator for our validation data
validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_dir.flow_from_directory(batch_size=batch_size, directory="/content/drive/MyDrive/plant_disease/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train", shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='categorical')
val_data_gen = valid_dir.flow_from_directory(batch_size=batch_size,directory="/content/drive/MyDrive/plant_disease/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",target_size=(IMG_HEIGHT, IMG_WIDTH),class_mode='categorical')

Plotting the images

In [ ]:
sample_training_images, _ = next(train_data_gen)
def plotImages(imagesArr):
  fig, axes = plt.subplots(1,5,figsize = (20,20))
  axes  = axes.flatten()
  for img ,ax in zip(imagesArr,axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()
plotImages(sample_training_images)


Creating the model using Sequential.

In [ ]:
model = Sequential()
model.add(Conv2D(16,3, padding ='same', activation = 'relu',input_shape = (IMG_HEIGHT,IMG_WIDTH,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64,3, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))

model.add(Dense(1))

In [ ]:

model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_data_gen, steps_per_epoch=64, epochs=10,validation_data=val_data_gen,validation_steps=64)


In [ ]:
h = history.history
print(h.keys())
print(h.values())

testing an image

In [ ]:
img = keras.preprocessing.image.load_img(
    "/content/drive/MyDrive/plant_disease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/Orange___Haunglongbing_(Citrus_greening)/0342765a-469b-4287-bdac-2c48875cc7f5___CREC_HLB 7706.JPG", target_size=(180,180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent cat and %.2f percent dog."
    % (100 * (1 - score), 100 * score)
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs_range = range(10)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Model Building for using the model Model.

In [ ]:
t_img, label = train.next()


In [ ]:
t_img.shape

In [ ]:
def plotImage(imgarr, label):
  for im, l in zip(imgarr, label):
    plt.figure(figsize = (5,5))
    plt.imshow(im)
    plt.show()

In [ ]:
plotImage(t_img[:3],label[:3])

In [ ]:
import keras

In [ ]:
base_model = VGG19(input_shape = (256,256,3), include_top = False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units = 38, activation = 'softmax')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

#Early Stopping and Model Check Point

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 3, verbose = 1)

mc = ModelCheckpoint(filepath = "best_model.h5", monitor='val_accuracy', min_delta = 0.01, patience =3, verbose = 1, save_best_only = True)

In [ ]:
cb = [es,mc]

In [ ]:
his = model.fit_generator(train,steps_per_epoch = 16, verbose = 1, callbacks = cb, validation_data = valid, validation_steps=16 )

In [ ]:
h = his.history
print(h.keys())
print(h.values())

In [ ]:
acc = h['accuracy']
val_acc = h['val_accuracy']
loss=h['loss']
val_loss=h['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()